# Hand Gesture Detection System
---

## 1. Download Dataset from Kaggle

In [ ]:
import kagglehub
from google.colab import drive
import shutil
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import random
import os
from collections import Counter
import seaborn as sns

---

## 2. Download Dataset Files

In [ ]:
# Download latest version
src_path = kagglehub.dataset_download("marusagar/hand-gesture-detection-system")

print("Path to dataset files:", src_path)

---

## 3. Mount Google Drive

In [ ]:
drive.mount('/content/drive')

---

## 4. Set Data Directory

In [ ]:
DATA_DIR = "/content/drive/MyDrive/Datasets/hand-gesture-detection_dataset"

---

## 5. Data Organization Setup

In [ ]:
# Paths
source_root = Path(src_path)
target_root = Path(DATA_DIR)

train_file = pd.read_csv(source_root / "train.csv")
val_file = pd.read_csv(source_root / "val.csv")

train_file.columns=['Image Name']
val_file.columns=['Image Name']

# Class folders
classes = {0: "Left_Swipe_new",
           1: "Right_Swipe_new",
           2: "Stop_new",
           3: "Thumbs_Down_new",
           4: "Thumbs_Up_new"}

def organize_data(csv_file, src_folder_name, target_subfolder):
    # Make sure target class folders exist
    for class_name in classes.values():
        (target_root / target_subfolder / class_name[:-4]).mkdir(parents=True, exist_ok=True)

    for _, row in csv_file.iterrows():
        folder_name = row['Image Name'][:row['Image Name'].find(";")]
        class_id = row['Image Name'][-1]
        class_name = classes[int(class_id)][:-4]

        src_folder = source_root / src_folder_name / src_folder_name / folder_name
        dest_folder = target_root / target_subfolder / class_name / folder_name

        if src_folder.exists():
            shutil.copytree(src_folder, dest_folder)
            print(f"Copied {folder_name} to {target_subfolder}/{class_name}")
        else:
            print(f"Folder {folder_name} not found in {src_folder_name}")

---

## 6. Organize Train and Validation Data

In [ ]:
#Organize train data
organize_data(train_file, "train", "train")

# Organize validation data
organize_data(val_file, "val", "val")

---

## 7. Create Test Split from Training Data

In [ ]:
# Paths
train_dir = Path( f"{DATA_DIR}/train")
test_dir  = Path( f"{DATA_DIR}/test" )

# Make sure test directory exists
test_dir.mkdir(parents=True, exist_ok=True)

# For each class folder inside train
for class_folder in train_dir.iterdir():
    if class_folder.is_dir():
        # Create corresponding class folder inside test
        target_class_folder = test_dir / class_folder.name
        target_class_folder.mkdir(parents=True, exist_ok=True)

        # List all samples (folders) in this class
        samples = [f for f in class_folder.iterdir() if f.is_dir()]

        # Take first 4 samples
        samples_to_move = samples[:4]

        for sample_folder in samples_to_move:
            dest = target_class_folder / sample_folder.name
            shutil.move(sample_folder, dest)
            print(f"moved {sample_folder.name} to test/{class_folder.name}")

---

## 8. Visualize Training Set Class Distribution

In [ ]:
# Count images per class
train_path = f"{DATA_DIR}/train"
train_counts = Counter([
    label for folder in os.listdir(train_path)
    if os.path.isdir(os.path.join(train_path, folder))
    for label in [folder] * len(os.listdir(os.path.join(train_path, folder)))
])

# Create DataFrame
df = pd.DataFrame(train_counts.items(), columns=['Class', 'Count'])
df = df.sort_values(by='Count', ascending=False)

# Plot
plt.figure(figsize=(9, 5))
ax = sns.barplot(data=df, x='Class', y='Count', palette='viridis')
plt.title('Sample Count per Class – Train Set')

# Add count labels inside bars
for p in ax.patches:
    height = p.get_height()
    ax.text(
        p.get_x() + p.get_width() / 2,
        height / 2,
        f'{int(height)}',
        ha='center', va='center',
        color='white', fontsize=12
    )
plt.tight_layout()
plt.show()

---

## 9. Create Sample Visualization Function

In [ ]:
def show_random_sample_frames(dataset_root, frame_count=30, step=1, cols=6, frame_size=(224,224)):
    dataset_root = Path(dataset_root)

    # Get all class folders
    class_folders = [folder for folder in dataset_root.iterdir() if folder.is_dir()]
    if not class_folders:
        print("No class folders found.")
        return

    # Pick a random class folder
    selected_class = random.choice(class_folders)

    # Pick a random sample folder inside that class
    sample_folders = [f for f in selected_class.iterdir() if f.is_dir()]
    if not sample_folders:
        print(f"No sample folders found in class {selected_class.name}")
        return

    selected_sample = random.choice(sample_folders)

    print(f"Class: {selected_class.name} | Sample: {selected_sample.name}")

    # Get image frames
    frames = sorted(selected_sample.glob('*.png'))[::step][:frame_count]
    if not frames:
        print("No frames found in selected sample.")
        return

    rows = max(1, (len(frames) + cols - 1) // cols)
    plt.figure(figsize=(cols * 2, rows * 2))

    for i, frame_path in enumerate(frames):
        img = cv2.imread(str(frame_path))
        if img is None:
            continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, frame_size)

        plt.subplot(rows, cols, i + 1)
        plt.imshow(img)
        plt.axis('off')

    plt.tight_layout()
    plt.show()

---

## 10. Visualize Random Samples

In [ ]:
# Example usage:
dataset_path = f"{DATA_DIR}/train"
show_random_sample_frames(dataset_path)

---